In [2]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
import json
import math
from shapely.geometry import shape
import geopandas as gpd


In [3]:
# sensor list
url = "http://127.0.0.1:8000/api/collections/airqualitysensor/items/?f=geojson&skipGeometry=true&limit=10000&sensor_type=Biossido di Azoto"
r = requests.get(url)
json_stations = r.json()
gdf = gpd.GeoDataFrame.from_features(json_stations['features'])

In [12]:
gdf

,geometry,sensor_type,measurement_unit,station_id,station_name,altitude,province,comune,is_historical,date_start,date_stop,utm_north,utm_east,latitude,longitude,sensor_id
0,POINT (9.07824 45.57172),Biossido di Azoto,µg/m³,540,Garbagnate Milanese v. Villoresi,175.0,MI,Garbagnate Milanese,True,1992-12-11,2018-07-30,5046467.0000,506105.0000,45.5717,9.0782,5543
1,POINT (9.69815 45.15937),Biossido di Azoto,µg/m³,608,Codogno v.Trento,59.0,LO,Codogno,False,2000-01-13,None,5000892.0000,554872.0000,45.1594,9.6981,5611
2,POINT (9.02640 45.62736),Biossido di Azoto,µg/m³,554,Saronno v.Santuario,212.0,VA,Saronno,False,1997-03-28,None,5052646.0000,502058.0000,45.6274,9.0264,5557
3,POINT (8.74415 45.25473),Biossido di Azoto,µg/m³,707,Mortara v. Trento,108.0,PV,Mortara,True,2007-10-04,2021-05-12,5011280.0000,479925.0000,45.2547,8.7442,10321
4,POINT (8.87406 45.11350),Biossido di Azoto,µg/m³,682,Ferrera Erbognone ENI,88.0,PV,Ferrera Erbognone,False,2006-01-21,None,4995567.0000,490094.0000,45.1135,8.8741,9993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,POINT (9.64365 45.69104),Biossido di Azoto,µg/m³,583,Bergamo v.Meucci,249.0,BG,Bergamo,False,1989-06-01,None,5059922.0000,550116.0000,45.6910,9.6437,5586
162,POINT (9.27357 45.58026),Biossido di Azoto,µg/m³,674,Monza v.Machiavelli,162.0,MB,Monza,False,2005-10-21,None,5047450.0000,521343.0000,45.5803,9.2736,9877
163,POINT (11.17839 45.00514),Biossido di Azoto,µg/m³,699,Magnacavallo campo sportivo,11.0,MN,Magnacavallo,True,2006-11-29,2018-01-01,4985830.0000,671675.0000,45.0051,11.1784,10119
164,POINT (11.11692 45.03394),Biossido di Azoto,µg/m³,702,Pieve di Coriano v. Bugatte,16.0,MN,Pieve di Coriano,True,2006-11-29,2018-01-01,4988901.0000,666747.0000,45.0339,11.1169,10155


In [20]:
sensor_list = gdf.sensor_id.unique()
sensor_list

array([ 5543,  5611,  5557, 10321,  9993,  5596,  5575,  5963, 10179,
        5595,  5555,  5505,  5549, 10143,  6685, 20495,  5508,  5541,
        5516,  9999,  5534,  5576,  5955,  9928, 10475,  6898,  9969,
        9857,  5556,  5533,  9897,  5552,  6826,  5566,  5591,  5570,
        6798,  5568,  5514,  5520, 10005, 12579,  5592,  5506, 10407,
        5597,  5604,  5563,  5545,  5532,  5961, 30160,  5601,  5567,
        5565, 10131, 10079,  5958,  5610,  5512,  6730,  5600, 10286,
        5507,  5603, 10108,  5559,  6871,  5950,  5546,  6880,  6761,
        5572,  5510,  5548,  6852,  5573,  5513,  5589,  5561,  5504,
       10097, 20443,  5579,  6744,  6885,  5593,  5590,  5540,  6701,
        5550,  9805,  9852, 10331, 10458,  5538,  5580,  6889, 10507,
       10452,  5560,  6841,  9802, 20491,  5522,  5598, 12017,  5587,
        5948,  5605,  5542, 10581,  5574,  5578,  5599, 20034, 10038,
        5528,  6594,  5551,  5953,  5577,  5554,  5569,  5517,  5521,
        5965, 10326,

In [63]:
date_filter = "2022-01-01/2022-12-31"
limit = "100000"
skip_geometry = "true"
f = "geojson"
root = "http://127.0.0.1:8000/api/collections/airqualitymeasurement/items/?"

complete_gdf = None
for sensor in sensor_list:
    sensor_obj = gdf.loc[gdf['sensor_id'] == sensor]
    geom = sensor_obj.geometry
    next_link = f'{root}f={f}&skipGeometry={skip_geometry}&limit={limit}&datetime={date_filter}&sensor_id={sensor}'
    station_gdf = None
    while next_link is not None:
        print(f'next_link {next_link}')
        r = requests.get(next_link)
        json_measurements = r.json()
        if station_gdf is None:
            station_gdf = gpd.GeoDataFrame.from_features(json_measurements['features'])
        else:
            station_gdf = pd.concat([station_gdf, gpd.GeoDataFrame.from_features(json_measurements['features'])])

        # iterate links
        next_link = None
        for link in json_measurements['links']:
            if link['rel'] == "next":
                next_link = link['href']

    #insert the geometry column
    station_gdf = station_gdf.assign(location=sensor_obj['geometry'].values[0])
    station_gdf = station_gdf.set_geometry( 'location' )
    
    if complete_gdf is None:
        complete_gdf = station_gdf.copy()
    else:
        complete_gdf = pd.concat([complete_gdf, station_gdf.copy()])

complete_gdf = complete_gdf.drop(columns=['geometry'])
complete_gdf


next_link http://127.0.0.1:8000/api/collections/airqualitymeasurement/items/?f=geojson&skipGeometry=true&limit=100000&datetime=2022-01-01/2022-12-31&sensor_id=5543
next_link http://127.0.0.1:8000/api/collections/airqualitymeasurement/items/?f=geojson&skipGeometry=true&limit=100000&datetime=2022-01-01/2022-12-31&sensor_id=5611
next_link http://127.0.0.1:8000/api/collections/airqualitymeasurement/items/?f=geojson&skipGeometry=true&limit=100000&datetime=2022-01-01/2022-12-31&sensor_id=5557
next_link http://127.0.0.1:8000/api/collections/airqualitymeasurement/items/?f=geojson&skipGeometry=true&limit=100000&datetime=2022-01-01/2022-12-31&sensor_id=10321
next_link http://127.0.0.1:8000/api/collections/airqualitymeasurement/items/?f=geojson&skipGeometry=true&limit=100000&datetime=2022-01-01/2022-12-31&sensor_id=9993
next_link http://127.0.0.1:8000/api/collections/airqualitymeasurement/items/?f=geojson&skipGeometry=true&limit=100000&datetime=2022-01-01/2022-12-31&sensor_id=5596
next_link http:

,location,date,value,sensor_id
0,POINT (9.69815 45.15937),2022-03-21T19:00:00,63.8000,5611.0
1,POINT (9.69815 45.15937),2022-01-01T00:00:00,30.0000,5611.0
2,POINT (9.69815 45.15937),2022-01-01T01:00:00,29.0000,5611.0
3,POINT (9.69815 45.15937),2022-01-01T02:00:00,26.5000,5611.0
4,POINT (9.69815 45.15937),2022-01-01T03:00:00,27.6000,5611.0
...,...,...,...,...
8695,POINT (9.27357 45.58026),2022-12-30T23:00:00,35.2000,9877.0
8696,POINT (9.27357 45.58026),2022-12-31T00:00:00,33.2000,9877.0
8697,POINT (9.27357 45.58026),2022-07-06T22:00:00,8.8000,9877.0
8698,POINT (9.27357 45.58026),2022-07-06T23:00:00,11.0000,9877.0


In [72]:
lat_lng_copy = complete_gdf.copy()

In [73]:
lat_lng_copy['lat'] = lat_lng_copy['location'].y
lat_lng_copy['lng'] = lat_lng_copy['location'].x
lat_lng_copy = lat_lng_copy[['lat', 'lng', 'date', 'value', 'sensor_id']]
lat_lng_copy

,lat,lng,date,value,sensor_id
0,45.159373,9.698148,2022-03-21T19:00:00,63.8000,5611.0
1,45.159373,9.698148,2022-01-01T00:00:00,30.0000,5611.0
2,45.159373,9.698148,2022-01-01T01:00:00,29.0000,5611.0
3,45.159373,9.698148,2022-01-01T02:00:00,26.5000,5611.0
4,45.159373,9.698148,2022-01-01T03:00:00,27.6000,5611.0
...,...,...,...,...,...
8695,45.580263,9.273573,2022-12-30T23:00:00,35.2000,9877.0
8696,45.580263,9.273573,2022-12-31T00:00:00,33.2000,9877.0
8697,45.580263,9.273573,2022-07-06T22:00:00,8.8000,9877.0
8698,45.580263,9.273573,2022-07-06T23:00:00,11.0000,9877.0


In [78]:
#complete_gdf.to_file("NO2_lombardia_2022.geojson", driver="GeoJSON")
#complete_gdf.to_csv("NO2_lombardia_2022.csv")
lat_lng_copy.to_csv("NO2_lombardia_2022_lat_lng.csv", index=False)